In [ ]:
import pandas as pd
import numpy as np

In [142]:
data1 = pd.read_excel("extract2.xlsx")
data1.drop(index=0, inplace=True)
data1.drop(columns=['PLY_NO', 'PYG_GST', 'APR_FLG', 'BSN_USE', 'BLD_CLM_CNT'], inplace=True)   # dropping cols with same values throughout
data1.reset_index(drop=True, inplace=True)
data1

,PLY_PRD,CST_OCP,TIR_PRM_AM,UCP_DY,UCP_RSN,OWS_DTL,SUM_INU,PRM_AM,PRM_WTH_IPT,RSK_PRM_INC_CRD_SCR,...,TIR_RSK_PRM,ITM_LMT_VL,MAN_PRT_PRM,ACD_PRM,OBL_PRM,PRV_ISN_BLD,PRV_ISN_CNE,PPT_ELG,RSK_PRM_OVR_TAX_SCG,CTS_CLM_CNT
0,DC1,RETI,0,30,Pendingsale(deceased),OR,NaN,NaN,NaN,NaN,...,0,550000,NaN,NaN,NaN,9,9,1,15.68,NaN
1,DC1,OTHR,195.2,30,NaN,OR,500000,174.29,195.2,162.41,...,159.51,99999999,NaN,NaN,NaN,5,5,1,34.8,0
2,DC1,OTHR,NaN,30,NaN,OR,0,39.6,NaN,NaN,...,NaN,99999999,1.789269,0.515735,0,7,7,1,77.99,0
3,DC1,OTHR,0,30,Pendingsale(deceased),UR,0,0,0,0,...,0,100000,NaN,NaN,NaN,0,2,1,27.13,0
4,DC1,RETI,0,30,NaN,OR,0,0,0,NaN,...,0,75000,0.688533,0.111698,0.068097,99,9,1,24.39,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71,DC1,RETI,0,30,NaN,OR,0,0,0,NaN,...,0,1000000,4.24824,0.3502,0.33187,99,4,1,31.32,0
72,DC1,OTHR,0,30,Pendingsale(deceased),FR,NaN,NaN,NaN,NaN,...,38.92,100000000,NaN,NaN,NaN,0,0,1,57.67,0
73,DC1,RETI,0,30,NaN,OR,NaN,NaN,NaN,NaN,...,0,100000,NaN,NaN,NaN,99,99,NaN,41.34,0
74,DC1,RETI,27.59,30,NaN,OR,2000,24.63,27.59,NaN,...,11.7,5000,9.31995,0.39542,0,99,0,NaN,27.06,0


In [143]:
print("Column, Missing Values, Unique Values")
for col in data1.columns:
    print(f"{col}, {data1[col].isna().sum()}, {data1[col].nunique()}")

Column, Missing Values, Unique Values
PLY_PRD, 0, 2
CST_OCP, 20, 3
TIR_PRM_AM, 8, 24
UCP_DY, 18, 2
UCP_RSN, 42, 1
OWS_DTL, 18, 4
SUM_INU, 37, 10
PRM_AM, 37, 17
PRM_WTH_IPT, 40, 15
RSK_PRM_INC_CRD_SCR, 64, 8
RSK_PRM_NO_CRD_SCR, 64, 8
CVR_IND, 10, 2
CVR_TYP_LMT, 60, 6
CVR_SEL, 45, 2
TIR_LMT_VL, 25, 11
TIR_RSK_PRM, 3, 30
ITM_LMT_VL, 3, 20
MAN_PRT_PRM, 42, 31
ACD_PRM, 42, 26
OBL_PRM, 42, 22
PRV_ISN_BLD, 6, 11
PRV_ISN_CNE, 6, 11
PPT_ELG, 25, 1
RSK_PRM_OVR_TAX_SCG, 0, 76
CTS_CLM_CNT, 35, 2


In [144]:
categorical_cols = ["CST_OCP", "UCP_DY", "OWS_DTL", "CVR_IND", "CVR_SEL", "CTS_CLM_CNT"]

# Replace string indicators of missingness with NaN
data1.replace(["", "NA", "None"], np.nan, inplace=True)

for col in categorical_cols:
    if col in data1.columns:
        choices = data1[col].dropna().unique()
        data1[col] = data1[col].apply(lambda x: np.random.choice(choices) if pd.isna(x) else x)

# standard_fill_cols
data1["UCP_RSN"] = data1["UCP_RSN"].fillna("Unknown")
data1["PPT_ELG"] = data1["PPT_ELG"].fillna(0)

C:\Users\niksh\AppData\Local\Temp\ipykernel_22252\3220750870.py:4: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data1.replace(["", "NA", "None"], np.nan, inplace=True)


In [145]:
categorical_cols.append("UCP_RSN")
categorical_cols.append("PPT_ELG")
categorical_cols.append("PLY_PRD")

rem_cols = data1.columns.difference(categorical_cols).tolist()

from scipy.stats import norm

for col in rem_cols:
    if col in data1.columns:
        non_null_values = data1[col].dropna().values
        if len(non_null_values) == 0:
            continue

        # Fit a normal distribution
        mu, sigma = norm.fit(non_null_values)

        n_missing = data1[col].isna().sum()
        # Generate samples
        sampled_values = np.random.normal(mu, sigma, n_missing)

        # Ensure no negatives — resample those that are < 0
        while np.any(sampled_values < 0):
            neg_idx = sampled_values < 0
            sampled_values[neg_idx] = np.random.normal(mu, sigma, np.sum(neg_idx))

        data1.loc[data1[col].isna(), col] = sampled_values

In [146]:
data1

,PLY_PRD,CST_OCP,TIR_PRM_AM,UCP_DY,UCP_RSN,OWS_DTL,SUM_INU,PRM_AM,PRM_WTH_IPT,RSK_PRM_INC_CRD_SCR,...,TIR_RSK_PRM,ITM_LMT_VL,MAN_PRT_PRM,ACD_PRM,OBL_PRM,PRV_ISN_BLD,PRV_ISN_CNE,PPT_ELG,RSK_PRM_OVR_TAX_SCG,CTS_CLM_CNT
0,DC1,RETI,0.000000,30.0,Pendingsale(deceased),OR,32575.971658,4.382008,52.703553,10.154968,...,0.000000,550000.0,2.168202,0.253472,0.526036,9.0,9.0,1.0,15.68,1.0
1,DC1,OTHR,195.200000,30.0,Unknown,OR,500000.000000,174.290000,195.200000,162.410000,...,159.510000,99999999.0,4.807503,0.181902,0.486737,5.0,5.0,1.0,34.80,0.0
2,DC1,OTHR,11.055799,30.0,Unknown,OR,0.000000,39.600000,72.766263,112.673521,...,40.841135,99999999.0,1.789269,0.515735,0.000000,7.0,7.0,1.0,77.99,0.0
3,DC1,OTHR,0.000000,30.0,Pendingsale(deceased),UR,0.000000,0.000000,0.000000,0.000000,...,0.000000,100000.0,7.644339,0.355045,0.356815,0.0,2.0,1.0,27.13,0.0
4,DC1,RETI,0.000000,30.0,Unknown,OR,0.000000,0.000000,0.000000,96.586864,...,0.000000,75000.0,0.688533,0.111698,0.068097,99.0,9.0,1.0,24.39,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71,DC1,RETI,0.000000,30.0,Unknown,OR,0.000000,0.000000,0.000000,93.117957,...,0.000000,1000000.0,4.248240,0.350200,0.331870,99.0,4.0,1.0,31.32,0.0
72,DC1,OTHR,0.000000,30.0,Pendingsale(deceased),FR,468988.071161,56.004511,43.160180,52.154636,...,38.920000,100000000.0,7.726063,0.225000,0.035774,0.0,0.0,1.0,57.67,0.0
73,DC1,RETI,0.000000,30.0,Unknown,OR,220597.591091,75.057272,102.606125,78.527447,...,0.000000,100000.0,0.903651,0.163108,0.527955,99.0,99.0,0.0,41.34,0.0
74,DC1,RETI,27.590000,30.0,Unknown,OR,2000.000000,24.630000,27.590000,2.515445,...,11.700000,5000.0,9.319950,0.395420,0.000000,99.0,0.0,0.0,27.06,0.0


In [149]:
from scipy.stats import gaussian_kde

def scale_dataframe(data, target_rows, categorical_cols, numeric_cols, random_state=None):
    """
    Scale up a mixed-type DataFrame to target_rows by synthetic resampling 
    while preserving per-column data distributions.

    Parameters
    ----------
    data : pd.DataFrame
        Original DataFrame (no NaN values).
    target_rows : int
        Desired total number of rows in the scaled dataset.
    categorical_cols : list
        List of column names that are categorical/string type.
    numeric_cols : list
        List of column names that are continuous numeric type.
    random_state : int, optional
        Random seed for reproducibility.

    Returns
    -------
    pd.DataFrame
        New DataFrame with 'target_rows' rows, synthetically generated.
    """
    np.random.seed(random_state)
    new_data = pd.DataFrame()

    # --- Handle categorical columns ---
    for col in categorical_cols:
        if col not in data.columns:
            continue
        col_data = data[col]
        # Compute probability of each category
        probs = col_data.value_counts(normalize=True)
        # Sample categories according to observed frequency
        sampled = np.random.choice(probs.index, size=target_rows, p=probs.values)
        new_data[col] = sampled

    # --- Handle numeric columns ---
    for col in numeric_cols:
        if col not in data.columns:
            continue
        col_data = data[col].values
        # Fit Kernel Density Estimation (KDE)
        kde = gaussian_kde(col_data)
        # Generate synthetic samples
        sampled = kde.resample(target_rows).reshape(-1)
        # Clip to observed range (optional, ensures realism)
        sampled = np.clip(sampled, col_data.min(), col_data.max())
        new_data[col] = sampled

    # Preserve column order as in the original DataFrame
    new_data = new_data[data.columns]

    return new_data.reset_index(drop=True)

# Generate scaled data
scaled_data = scale_dataframe(data1, target_rows=20000,
                              categorical_cols=categorical_cols,
                              numeric_cols=rem_cols,
                              random_state=42)

In [150]:
scaled_data

,PLY_PRD,CST_OCP,TIR_PRM_AM,UCP_DY,UCP_RSN,OWS_DTL,SUM_INU,PRM_AM,PRM_WTH_IPT,RSK_PRM_INC_CRD_SCR,...,TIR_RSK_PRM,ITM_LMT_VL,MAN_PRT_PRM,ACD_PRM,OBL_PRM,PRV_ISN_BLD,PRV_ISN_CNE,PPT_ELG,RSK_PRM_OVR_TAX_SCG,CTS_CLM_CNT
0,DC2,OTHR,36.950781,30.0,Unknown,OR,233749.324121,14.031104,0.000000,48.148990,...,0.000000,1.195014e+07,7.071151,0.577860,0.558543,97.424172,15.090206,1.0,12.636697,0.0
1,DC1,HWIF,16.785905,30.0,Unknown,OR,750000.000000,57.018029,0.000000,51.729955,...,0.000000,9.471779e+07,1.173795,0.079702,0.600235,99.000000,116.401152,0.0,43.551814,0.0
2,DC1,RETI,13.653650,30.0,Pendingsale(deceased),OR,66406.128488,9.979907,111.539685,161.831381,...,6.158990,1.000000e+08,10.129723,0.110965,0.492970,99.000000,97.974447,1.0,12.846793,0.0
3,DC2,OTHR,0.000000,30.0,Unknown,OR,75879.242639,80.719966,2.114125,179.716754,...,268.257836,7.081339e+06,2.832058,0.003882,0.466926,99.000000,0.000000,1.0,22.438149,0.0
4,DC1,OTHR,4.244899,30.0,Unknown,OR,0.000000,0.000000,0.000000,52.643709,...,29.421468,1.033186e+07,12.159161,0.297659,0.776566,16.455473,89.079765,1.0,42.475082,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,DC1,RETI,0.000000,30.0,Pendingsale(deceased),FR,254042.427230,31.330103,80.686998,163.925896,...,3.880333,9.286572e+07,1.279818,0.586647,0.289192,0.000000,28.855785,0.0,64.710968,0.0
19996,DC1,OTHR,0.000000,30.0,Unknown,OR,10622.442605,0.000000,0.000000,68.439643,...,74.430706,1.408699e+07,12.193228,0.023413,0.172116,23.956453,2.795530,1.0,19.538796,0.0
19997,DC2,OTHR,18.931007,30.0,Unknown,OR,229089.020292,0.000000,328.100983,7.424533,...,90.342444,1.000000e+08,10.934210,0.145899,0.522996,99.000000,12.516599,1.0,27.698942,0.0
19998,DC1,OTHR,0.000000,30.0,Unknown,OR,0.000000,31.436911,24.319584,72.058928,...,20.527527,1.467108e+07,0.716785,0.463595,0.234709,10.761742,0.000000,0.0,85.122474,0.0


In [154]:
scaled_data.to_csv("scaled_extract2.csv", index=False)